In [233]:
import csv
from pathlib import Path
import pandas as pd
from pandas import DataFrame
import os
from dotenv import load_dotenv
from datetime import datetime
from datetime import date
from datetime import timedelta
import requests
import json

import matplotlib.pyplot as plt
import hvplot.pandas
import plotly.express as px
import panel as pn
from panel.interact import interact
from panel import widgets
import numpy as np

%matplotlib inline

# Initialize the Panel Extensions (for Plotly)
pn.extension('plotly')

# Input Portfolio
Read in the input portfolio CSV file and output a portfolio dataframe
* Inputs:
    * Portfolio CSV file
    * Format: Ticker, Transaction (B=buy, S=Sell), Date, Transaction Price, Number of Shares
    * Assumptions: Date unsorted, no '$' sign, no commas
* Outputs:
    * Portfolio database
    * Format: Ticker, Transaction (B=buy, S=Sell), Date, Transaction Price, Number of Shares, Transaction Cost
    * Database sorted by ascending date

In [2]:
# Set the path for the CSV file
input_file_path = Path("TestPortfolio.csv")

In [3]:
#Create empty list to store row data from CSV
portfolio_transactions = []

In [4]:
#Open the CSV file; read in row data; calculate transaction cost per transacation
with open(input_file_path, 'r') as csvfile:
    #Set reader object
    csvreader = csv.reader(csvfile, delimiter = ',')
 
    #Read the header
    csv_header = next(csvreader)
    
    #Create new column
    csv_header.append("Transaction Cost")
    portfolio_transactions.append(csv_header)

    #Iterate through the data 
    for row in csvreader:
        #Calculate transaction cost
        cost = float(row[3]) * float(row[4])
        
        #Append transaction cost to list
        row.append(cost)
        portfolio_transactions.append(row)

In [5]:
#Convert transaction list to dataframe
portfolio_df = pd.DataFrame(portfolio_transactions, columns=['Ticker', 'Transaction', 'Date', 'Transaction Price', 'Number of shares', 'Transaction Cost'])
#Drop first row (duplicate header)
portfolio_df=portfolio_df.drop(portfolio_df.index[0])
portfolio_df

,Ticker,Transaction,Date,Transaction Price,Number of shares,Transaction Cost
1,AAPL,B,3/5/12,77.8,100,7780
2,AAPL,B,4/1/13,60.57,100,6057
3,TSLA,B,9/23/13,189.9,100,18990
4,TSLA,S,11/18/13,122.38,75,9178.5
5,TSLA,B,5/9/16,208,30,6240
6,GOOG,B,8/7/17,915.39,10,9153.9
7,AAPL,S,3/16/20,240,50,12000


In [6]:
#Convert Date to datetime and set as index
portfolio_df['Date']=pd.to_datetime(portfolio_df['Date'])
portfolio_df.set_index(['Date'], inplace=True)
portfolio_df

,Ticker,Transaction,Transaction Price,Number of shares,Transaction Cost
Date,,,,,
2012-03-05,AAPL,B,77.8,100,7780
2013-04-01,AAPL,B,60.57,100,6057
2013-09-23,TSLA,B,189.9,100,18990
2013-11-18,TSLA,S,122.38,75,9178.5
2016-05-09,TSLA,B,208,30,6240
2017-08-07,GOOG,B,915.39,10,9153.9
2020-03-16,AAPL,S,240,50,12000


In [7]:
# Sort dataframe in ascending order
portfolio_df.sort_index(inplace=True)
portfolio_df

,Ticker,Transaction,Transaction Price,Number of shares,Transaction Cost
Date,,,,,
2012-03-05,AAPL,B,77.8,100,7780
2013-04-01,AAPL,B,60.57,100,6057
2013-09-23,TSLA,B,189.9,100,18990
2013-11-18,TSLA,S,122.38,75,9178.5
2016-05-09,TSLA,B,208,30,6240
2017-08-07,GOOG,B,915.39,10,9153.9
2020-03-16,AAPL,S,240,50,12000


# Pull Stock Data
Using the portfolio database, determine the number of unique stocks and pull the stock history.
* Inputs:
    * Portfolio database (portfolio_df)
* Outputs:
    * Ticker Stat database (ticker_stats_df) (Tickers as Rows)
    * Transposed Ticker Stat Database (ticker_stats_df_T) (Tickers as Columns)
    * Dailiy Closing Price database (closing_df)

In [8]:
# Set environment variables from the .env file
# rel_path = Path('../../Python') / '.env'
# load_dotenv(rel_path)
load_dotenv("/Users/heenaroy/Desktop/.env")

True

In [9]:
# Grab API Key for https://financialmodelingprep.com
stock_key = os.getenv("FINANCIAL_MODEL_KEY")

In [10]:
# Create empty list for stock tickers
stock_list=[]

In [11]:
# Determine unique stocks
group=portfolio_df.groupby('Ticker')
# Store unique stocks to stock list
group.groups.keys()
stock_list=list(group.groups)
stock_list

['AAPL', 'GOOG', 'TSLA']

In [12]:
#Create new dataframe resetting date index
portfolio2_df=portfolio_df.reset_index()
portfolio2_df

,Date,Ticker,Transaction,Transaction Price,Number of shares,Transaction Cost
0,2012-03-05,AAPL,B,77.8,100,7780
1,2013-04-01,AAPL,B,60.57,100,6057
2,2013-09-23,TSLA,B,189.9,100,18990
3,2013-11-18,TSLA,S,122.38,75,9178.5
4,2016-05-09,TSLA,B,208,30,6240
5,2017-08-07,GOOG,B,915.39,10,9153.9
6,2020-03-16,AAPL,S,240,50,12000


In [13]:
#Find the min dates per tikcer
min_stock_dates=portfolio2_df.groupby('Ticker').min()
min_stock_dates

,Date,Transaction,Transaction Price,Number of shares,Transaction Cost
Ticker,,,,,
AAPL,2012-03-05,B,240,100,6057.0
GOOG,2017-08-07,B,915.39,10,9153.9
TSLA,2013-09-23,B,122.38,100,6240.0


In [14]:
#Store yesterday's date
yesterday = date.today() - timedelta(days=1)
yesterday = yesterday.strftime("%Y-%m-%d")
yesterday

'2020-06-15'

In [15]:
#Define Main Loop Variables
stock_dict={}            # Create dictionary to store stock data
count=0                  # Create counter as index for looping through stock list

#For each ticker in the stock list, pull the stock data
for ticker in stock_list:
    #Define new dictionary key set per ticker
    stock_dict[ticker]={
        'PE Ratio'            : 0,
        'Profit Margin'       : 0,
        'Cash'                : 0,
        'Beta'                : 0,
        'Last Dividend'       : 0,
        'Industry'            : 0,
        'Sector'              : 0,
        'Daily Closing Price' : 0,
        }
    
######
    #Get Beta, Last Div, Industry and Sector
    url = f"https://financialmodelingprep.com/api/v3/profile/{ticker}?apikey={stock_key}"
    requests.get(url).content
    parsed = json.loads(requests.get(url).content)
    
    #Store stock data in dictionary
    stock_dict[ticker]['Beta']=parsed[0]['beta']
    stock_dict[ticker]['Last Dividend']=parsed[0]['lastDiv']
    stock_dict[ticker]['Industry']=parsed[0]['industry']
    stock_dict[ticker]['Sector']=parsed[0]['sector']
    

######
    # Grab PE Ratio
    url = f"https://financialmodelingprep.com/api/v3/key-metrics/{ticker}?apikey={stock_key}"
    requests.get(url).content
    parsed = json.loads(requests.get(url).content)
    
    # Store PE Ratio in dictionary
    stock_dict[ticker]['PE Ratio']=parsed[0]['peRatio']
    
    
######    
    # Grab Profit Margin
    url = f"https://financialmodelingprep.com/api/v3/ratios/{ticker}?apikey={stock_key}"
    requests.get(url).content
    parsed = json.loads(requests.get(url).content)
    
    # Store Profit Margin in dictionary
    stock_dict[ticker]['Profit Margin']=parsed[0]['netProfitMargin']
    
   
###### 
    # Grab Cash
    url = f"https://financialmodelingprep.com/api/v3/balance-sheet-statement/{ticker}?apikey={stock_key}"
    requests.get(url).content
    parsed = json.loads(requests.get(url).content)
    
    # Store Profit Margin in dictionary
    stock_dict[ticker]['Cash']=parsed[0]['cashAndCashEquivalents']

     
######
    #Grab the first purchase date for current ticker
    datetime_obj=datetime.strptime(str(min_stock_dates['Date'][ticker]),"%Y-%m-%d %H:%M:%S")
    date_obj=datetime_obj.date()
    first_purchase_date=str(date_obj)
    
    # Grab the Daily Closing Prices between the first purchase date and yesterday
    url = f"https://financialmodelingprep.com/api/v3/historical-price-full/{ticker}?from={first_purchase_date}&to={yesterday}&apikey={stock_key}"
    requests.get(url).content
    parsed = json.loads(requests.get(url).content)
 
    ### Daily Stock Pull Variables
    count2=0                 # Create counter as index for looping through dictionary of stock data    
    daily_dict={}            # Create empty dictionary to store daily stock data before appending to daily_list_of_dict
    daily_list_of_dict = []  # Create empty list to store dictionary info from daily_dict before storing to stock_dict

    # Loop through historical data and grab daily closing date and price
    for i in parsed['historical']:
        daily_dict['Date'] = parsed['historical'][count2]['date']
        daily_dict['Price'] = parsed['historical'][count2]['close']
        daily_list_of_dict.append(daily_dict.copy())
        count2+=1

    #Store historical stock data to stock_dict dictionary
    stock_dict[ticker]['Daily Closing Price']=daily_list_of_dict
    
    
    #increment counter
    count+=1 

In [17]:
#Copy stock_dict so that main loop above doesn't have to be run multiple times
stock_dict_copy = stock_dict
stock_dict_copy

stock_dict_copy2=pd.DataFrame(stock_dict_copy)
stock_dict_copy2.to_csv('stock_dict_copy.csv')

In [18]:
# Create DataFrame showing Beta, P/E Ratio, Profit Margin, 
# Cash, and current Dividends per ticker
ticker_stats_df=DataFrame(stock_dict_copy)
ticker_stats_df.drop('Daily Closing Price', inplace=True)
ticker_stats_df

,AAPL,GOOG,TSLA
Beta,1.2285,1.02276,0.580101
Cash,48844000000,18498000000,6268000000
Industry,Consumer Electronics,Internet Content & Information,Auto Manufacturers
Last Dividend,3.08,0,0
PE Ratio,20.8135,14.6956,-131.582
Profit Margin,0.212381,0.212181,-0.035072
Sector,Technology,Communication Services,Consumer Cyclical


In [19]:
# Create a transpose of ticker_stats_df
ticker_stats_df_T=ticker_stats_df.T
ticker_stats_df_T=ticker_stats_df_T[['PE Ratio', 'Profit Margin', 'Cash', 'Beta', 'Last Dividend', 'Industry', 'Sector']]
ticker_stats_df_T

,PE Ratio,Profit Margin,Cash,Beta,Last Dividend,Industry,Sector
AAPL,20.8135,0.212381,48844000000,1.2285,3.08,Consumer Electronics,Technology
GOOG,14.6956,0.212181,18498000000,1.02276,0,Internet Content & Information,Communication Services
TSLA,-131.582,-0.035072,6268000000,0.580101,0,Auto Manufacturers,Consumer Cyclical


In [20]:
# Closing Data

In [21]:
# Create DataFrame showing Daily Closing Price per ticker
daily_closing_df=DataFrame(stock_dict_copy)
daily_closing_df.drop(['Beta', 'Cash', 'Industry', 'Last Dividend', 'PE Ratio', 'Profit Margin', 'Sector'], inplace=True)
daily_closing_df

,AAPL,GOOG,TSLA
Daily Closing Price,"[{'Date': '2020-06-15', 'Price': 342.99}, {'Da...","[{'Date': '2020-06-15', 'Price': 1419.85}, {'D...","[{'Date': '2020-06-15', 'Price': 990.9}, {'Dat..."


In [22]:
aapl_df = pd.DataFrame(daily_closing_df['AAPL'][0])
#aapl_df.rename(columns={'Date' : 'AAPL Closing Date', 'Price' : 'AAPL Closing Price'}, inplace=True)
aapl_df.rename(columns={'Price' : 'AAPL Closing Price'}, inplace=True)
aapl_df.head()

,Date,AAPL Closing Price
0,2020-06-15,342.99
1,2020-06-12,338.80
2,2020-06-11,335.90
3,2020-06-10,352.84
4,2020-06-09,343.99


In [23]:
goog_df = pd.DataFrame(daily_closing_df['GOOG'][0])
#goog_df.rename(columns={'Date' : 'GOOG Closing Date', 'Price' : 'GOOG Closing Price'}, inplace=True)
goog_df.rename(columns={'Price' : 'GOOG Closing Price'}, inplace=True)
goog_df.head()

,Date,GOOG Closing Price
0,2020-06-15,1419.85
1,2020-06-12,1413.18
2,2020-06-11,1403.84
3,2020-06-10,1465.85
4,2020-06-09,1456.16


In [24]:
tsla_df = pd.DataFrame(daily_closing_df['TSLA'][0])
#tsla_df.rename(columns={'Date' : 'TSLA Closing Date', 'Price' : 'TSLA Closing Price'}, inplace=True)
tsla_df.rename(columns={'Price' : 'TSLA Closing Price'}, inplace=True)
tsla_df.head()

,Date,TSLA Closing Price
0,2020-06-15,990.90
1,2020-06-12,935.28
2,2020-06-11,972.84
3,2020-06-10,1025.05
4,2020-06-09,940.67


In [25]:
closing_df = aapl_df.set_index('Date').join(goog_df.set_index('Date')).join(tsla_df.set_index('Date'))
closing_df.head()

,AAPL Closing Price,GOOG Closing Price,TSLA Closing Price
Date,,,
2020-06-15,342.99,1419.85,990.90
2020-06-12,338.80,1413.18,935.28
2020-06-11,335.90,1403.84,972.84
2020-06-10,352.84,1465.85,1025.05
2020-06-09,343.99,1456.16,940.67


# Insert Alex's Code

In [26]:
# Convert daily closing price data (stock_dict_copy['Daily Closing Price'])
# into a new dataframe (closing_price_df) with only Date, Ticker, and Close columns
date=[]
price=[]
stock_name=[]

for stock in stock_dict_copy:
    for d in stock_dict_copy[stock]['Daily Closing Price']:
        stock_name.append(stock)
        date.append(datetime.strptime(d['Date'],'%Y-%m-%d'))
        price.append(d['Price'])
        
closing_price_df= pd.DataFrame(list(zip(date, stock_name, price)), columns=['Date', 'Ticker', 'Close'])
closing_price_df

,Date,Ticker,Close
0,2020-06-15,AAPL,342.99
1,2020-06-12,AAPL,338.80
2,2020-06-11,AAPL,335.90
3,2020-06-10,AAPL,352.84
4,2020-06-09,AAPL,343.99
5,2020-06-08,AAPL,333.46
6,2020-06-05,AAPL,331.50
7,2020-06-04,AAPL,322.32
8,2020-06-03,AAPL,325.12
9,2020-06-02,AAPL,323.34


In [27]:
closing_price_df=closing_price_df.sort_values('Date')
closing_price_df

,Date,Ticker,Close
2083,2012-03-05,AAPL,76.17
2082,2012-03-06,AAPL,75.75
2081,2012-03-07,AAPL,75.81
2080,2012-03-08,AAPL,77.43
2079,2012-03-09,AAPL,77.88
2078,2012-03-12,AAPL,78.86
2077,2012-03-13,AAPL,81.16
2076,2012-03-14,AAPL,84.23
2075,2012-03-15,AAPL,83.65
2074,2012-03-16,AAPL,83.65


In [28]:
#Merge the input portfolio dataframe to the closing price dataframe
portfolio3_df = pd.merge(portfolio_df, closing_price_df, how='right', left_on=['Date', 'Ticker'], right_on=['Date', 'Ticker'])
portfolio3_df

,Date,Ticker,Transaction,Transaction Price,Number of shares,Transaction Cost,Close
0,2012-03-05,AAPL,B,77.8,100,7780,76.17
1,2013-04-01,AAPL,B,60.57,100,6057,61.27
2,2013-09-23,TSLA,B,189.9,100,18990,181.11
3,2013-11-18,TSLA,S,122.38,75,9178.5,121.58
4,2016-05-09,TSLA,B,208,30,6240,208.92
5,2017-08-07,GOOG,B,915.39,10,9153.9,929.36
6,2020-03-16,AAPL,S,240,50,12000,242.21
7,2012-03-06,AAPL,NaN,NaN,NaN,NaN,75.75
8,2012-03-07,AAPL,NaN,NaN,NaN,NaN,75.81
9,2012-03-08,AAPL,NaN,NaN,NaN,NaN,77.43


In [29]:
portfolio3_df=portfolio3_df.sort_values('Date')
portfolio3_df

,Date,Ticker,Transaction,Transaction Price,Number of shares,Transaction Cost,Close
0,2012-03-05,AAPL,B,77.8,100,7780,76.17
7,2012-03-06,AAPL,NaN,NaN,NaN,NaN,75.75
8,2012-03-07,AAPL,NaN,NaN,NaN,NaN,75.81
9,2012-03-08,AAPL,NaN,NaN,NaN,NaN,77.43
10,2012-03-09,AAPL,NaN,NaN,NaN,NaN,77.88
11,2012-03-12,AAPL,NaN,NaN,NaN,NaN,78.86
12,2012-03-13,AAPL,NaN,NaN,NaN,NaN,81.16
13,2012-03-14,AAPL,NaN,NaN,NaN,NaN,84.23
14,2012-03-15,AAPL,NaN,NaN,NaN,NaN,83.65
15,2012-03-16,AAPL,NaN,NaN,NaN,NaN,83.65


In [30]:
portfolio3_df['Number of shares']=portfolio3_df['Number of shares'].fillna(0)
portfolio3_df

,Date,Ticker,Transaction,Transaction Price,Number of shares,Transaction Cost,Close
0,2012-03-05,AAPL,B,77.8,100,7780,76.17
7,2012-03-06,AAPL,NaN,NaN,0,NaN,75.75
8,2012-03-07,AAPL,NaN,NaN,0,NaN,75.81
9,2012-03-08,AAPL,NaN,NaN,0,NaN,77.43
10,2012-03-09,AAPL,NaN,NaN,0,NaN,77.88
11,2012-03-12,AAPL,NaN,NaN,0,NaN,78.86
12,2012-03-13,AAPL,NaN,NaN,0,NaN,81.16
13,2012-03-14,AAPL,NaN,NaN,0,NaN,84.23
14,2012-03-15,AAPL,NaN,NaN,0,NaN,83.65
15,2012-03-16,AAPL,NaN,NaN,0,NaN,83.65


In [31]:
# Convert Number of shares and Transaction Price to float
portfolio3_df['Number of shares']=portfolio3_df['Number of shares'].astype(float)
portfolio3_df['Transaction Price']=portfolio3_df['Transaction Price'].astype(float)

In [32]:
# Calculate the net number of shares and total number of shares
portfolio3_df['Net Shares'] = portfolio3_df.apply(lambda row: row['Number of shares'] if row.Transaction == "B" else row['Number of shares'] * -1, axis=1)
portfolio3_df['Total Shares'] = portfolio3_df[['Net Shares','Ticker']].groupby('Ticker').cumsum()
portfolio3_df

,Date,Ticker,Transaction,Transaction Price,Number of shares,Transaction Cost,Close,Net Shares,Total Shares
0,2012-03-05,AAPL,B,77.8,100.0,7780,76.17,100.0,100.0
7,2012-03-06,AAPL,NaN,NaN,0.0,NaN,75.75,-0.0,100.0
8,2012-03-07,AAPL,NaN,NaN,0.0,NaN,75.81,-0.0,100.0
9,2012-03-08,AAPL,NaN,NaN,0.0,NaN,77.43,-0.0,100.0
10,2012-03-09,AAPL,NaN,NaN,0.0,NaN,77.88,-0.0,100.0
11,2012-03-12,AAPL,NaN,NaN,0.0,NaN,78.86,-0.0,100.0
12,2012-03-13,AAPL,NaN,NaN,0.0,NaN,81.16,-0.0,100.0
13,2012-03-14,AAPL,NaN,NaN,0.0,NaN,84.23,-0.0,100.0
14,2012-03-15,AAPL,NaN,NaN,0.0,NaN,83.65,-0.0,100.0
15,2012-03-16,AAPL,NaN,NaN,0.0,NaN,83.65,-0.0,100.0


In [33]:
# Calculate cum Daily Balances based on Total Shares and daily closing price
portfolio3_df['Daily Balance'] = portfolio3_df['Close']*portfolio3_df['Total Shares']
portfolio3_df

,Date,Ticker,Transaction,Transaction Price,Number of shares,Transaction Cost,Close,Net Shares,Total Shares,Daily Balance
0,2012-03-05,AAPL,B,77.8,100.0,7780,76.17,100.0,100.0,7617.00
7,2012-03-06,AAPL,NaN,NaN,0.0,NaN,75.75,-0.0,100.0,7575.00
8,2012-03-07,AAPL,NaN,NaN,0.0,NaN,75.81,-0.0,100.0,7581.00
9,2012-03-08,AAPL,NaN,NaN,0.0,NaN,77.43,-0.0,100.0,7743.00
10,2012-03-09,AAPL,NaN,NaN,0.0,NaN,77.88,-0.0,100.0,7788.00
11,2012-03-12,AAPL,NaN,NaN,0.0,NaN,78.86,-0.0,100.0,7886.00
12,2012-03-13,AAPL,NaN,NaN,0.0,NaN,81.16,-0.0,100.0,8116.00
13,2012-03-14,AAPL,NaN,NaN,0.0,NaN,84.23,-0.0,100.0,8423.00
14,2012-03-15,AAPL,NaN,NaN,0.0,NaN,83.65,-0.0,100.0,8365.00
15,2012-03-16,AAPL,NaN,NaN,0.0,NaN,83.65,-0.0,100.0,8365.00


In [35]:
portfolio3_df['Date']

0      2012-03-05
7      2012-03-06
8      2012-03-07
9      2012-03-08
10     2012-03-09
11     2012-03-12
12     2012-03-13
13     2012-03-14
14     2012-03-15
15     2012-03-16
16     2012-03-19
17     2012-03-20
18     2012-03-21
19     2012-03-22
20     2012-03-23
21     2012-03-26
22     2012-03-27
23     2012-03-28
24     2012-03-29
25     2012-03-30
26     2012-04-02
27     2012-04-03
28     2012-04-04
29     2012-04-05
30     2012-04-09
31     2012-04-10
32     2012-04-11
33     2012-04-12
34     2012-04-13
35     2012-04-16
          ...    
4467   2020-06-02
4468   2020-06-02
4469   2020-06-02
4470   2020-06-03
4471   2020-06-03
4472   2020-06-03
4475   2020-06-04
4473   2020-06-04
4474   2020-06-04
4476   2020-06-05
4477   2020-06-05
4478   2020-06-05
4479   2020-06-08
4480   2020-06-08
4481   2020-06-08
4483   2020-06-09
4482   2020-06-09
4484   2020-06-09
4486   2020-06-10
4485   2020-06-10
4487   2020-06-10
4489   2020-06-11
4488   2020-06-11
4490   2020-06-11
4492   202

In [40]:
portfolio3_df['Cumm Contribution'] = pd.DataFrame(portfolio3_df['Transaction Price'] * portfolio3_df['Net Shares'])
portfolio3_df

,Date,Ticker,Transaction,Transaction Price,Number of shares,Transaction Cost,Close,Net Shares,Total Shares,Daily Balance,Cumm Contribution
0,2012-03-05,AAPL,B,77.8,100.0,7780,76.17,100.0,100.0,7617.00,7780.0
7,2012-03-06,AAPL,NaN,NaN,0.0,NaN,75.75,-0.0,100.0,7575.00,NaN
8,2012-03-07,AAPL,NaN,NaN,0.0,NaN,75.81,-0.0,100.0,7581.00,NaN
9,2012-03-08,AAPL,NaN,NaN,0.0,NaN,77.43,-0.0,100.0,7743.00,NaN
10,2012-03-09,AAPL,NaN,NaN,0.0,NaN,77.88,-0.0,100.0,7788.00,NaN
11,2012-03-12,AAPL,NaN,NaN,0.0,NaN,78.86,-0.0,100.0,7886.00,NaN
12,2012-03-13,AAPL,NaN,NaN,0.0,NaN,81.16,-0.0,100.0,8116.00,NaN
13,2012-03-14,AAPL,NaN,NaN,0.0,NaN,84.23,-0.0,100.0,8423.00,NaN
14,2012-03-15,AAPL,NaN,NaN,0.0,NaN,83.65,-0.0,100.0,8365.00,NaN
15,2012-03-16,AAPL,NaN,NaN,0.0,NaN,83.65,-0.0,100.0,8365.00,NaN


In [108]:
portfolio3_df['Gains/Losses'] = portfolio3_df['Daily Balance'] - portfolio3_df['Transaction Cost']
portfolio3_df.tail()

,Date,Ticker,Transaction,Transaction Price,Number of shares,Transaction Cost,Close,Net Shares,Total Shares,Daily Balance,Cumm Contribution,Gains/Losses
4491,2020-06-12,GOOG,NaN,NaN,0.0,NaN,1413.18,-0.0,10.0,14131.8,NaN,NaN
4493,2020-06-12,TSLA,NaN,NaN,0.0,NaN,935.28,-0.0,55.0,51440.4,NaN,NaN
4495,2020-06-15,GOOG,NaN,NaN,0.0,NaN,1419.85,-0.0,10.0,14198.5,NaN,NaN
4494,2020-06-15,TSLA,NaN,NaN,0.0,NaN,990.90,-0.0,55.0,54499.5,NaN,NaN
4496,2020-06-15,AAPL,NaN,NaN,0.0,NaN,342.99,-0.0,150.0,51448.5,NaN,NaN


# Insert Heena's Code

In [347]:
# Define last closing price date

max_date=portfolio3_df["Date"].max()
max_date


Timestamp('2020-06-15 00:00:00')

In [348]:
# Create new dataframe with Current Values for Total Shares, Daily Balance, and Cumm Contribution

holdings_df=portfolio3_df[portfolio3_df.Date == max_date]
holdings_df=holdings_df.drop(columns=["Date", "Transaction", "Transaction Price", "Number of shares", "Transaction Cost", "Close", "Net Shares", "Gains/Losses"]).sort_values("Ticker")
holdings_df


,Ticker,Total Shares,Daily Balance,Cumm Contribution
4496,AAPL,150.0,51448.5,NaN
4495,GOOG,10.0,14198.5,NaN
4494,TSLA,55.0,54499.5,NaN


In [349]:
# Create new dataframe with Industry & Sector information by Ticker

sector_df=ticker_stats_df_T[["Industry", "Sector"]].reset_index()
sector_df.rename(columns={"index":"Ticker"}, inplace=True)
sector_df


,Ticker,Industry,Sector
0,AAPL,Consumer Electronics,Technology
1,GOOG,Internet Content & Information,Communication Services
2,TSLA,Auto Manufacturers,Consumer Cyclical


In [350]:
# Create new dataframe with Current Values and Industry/Sector information by Ticker

current_industry_holdings = pd.merge(holdings_df, sector_df, on="Ticker")
current_industry_holdings


,Ticker,Total Shares,Daily Balance,Cumm Contribution,Industry,Sector
0,AAPL,150.0,51448.5,NaN,Consumer Electronics,Technology
1,GOOG,10.0,14198.5,NaN,Internet Content & Information,Communication Services
2,TSLA,55.0,54499.5,NaN,Auto Manufacturers,Consumer Cyclical


In [380]:
# Create Pie Charts for Current Value, Contributions, and Total Shares by Ticker

holdings_contribution_pie=px.pie(current_industry_holdings, values="Cumm Contribution", names="Ticker", title="Contributions by Stock")
holdings_shares_pie=px.pie(current_industry_holdings, values="Total Shares", names="Ticker", title="Shares Owned by Stock")
holdings_value_pie=px.pie(current_industry_holdings, values="Daily Balance", names="Ticker", title="Current Value by Stock")


In [381]:
# Create new dataframe for Industry holdings

industry_holdings = current_industry_holdings.groupby("Industry").sum()
industry_holdings.reset_index(inplace=True)
industry_holdings


,Industry,Total Shares,Daily Balance,Cumm Contribution
0,Auto Manufacturers,55.0,54499.5,0.0
1,Consumer Electronics,150.0,51448.5,0.0
2,Internet Content & Information,10.0,14198.5,0.0


In [382]:
# Create Pie Charts for Current Value, Contributions, and Total Shares by Industry

industry_contribution_pie=px.pie(industry_holdings, values="Cumm Contribution", names="Industry", title="Contributions by Industry")
industry_shares_pie=px.pie(industry_holdings, values="Total Shares", names="Industry", title="Shares Owned by Industry")
industry_value_pie=px.pie(industry_holdings, values="Daily Balance", names="Industry", title="Current Value by Industry")


In [383]:
# Create new dataframe for Sector holdings

sector_holdings= current_industry_holdings.groupby("Sector").sum()
sector_holdings.reset_index(inplace=True)
sector_holdings


,Sector,Total Shares,Daily Balance,Cumm Contribution
0,Communication Services,10.0,14198.5,0.0
1,Consumer Cyclical,55.0,54499.5,0.0
2,Technology,150.0,51448.5,0.0


In [384]:
# Create Pie Charts for Current Value, Contributions, and Total Shares by Sector

sector_contribution_pie=px.pie(sector_holdings, values="Cumm Contribution", names="Sector", title="Contributions by Sector")
sector_shares_pie=px.pie(sector_holdings, values="Total Shares", names="Sector", title="Shares Owned by Sector")
sector_value_pie=px.pie(sector_holdings, values="Daily Balance", names="Sector", title="Current Value by Sector")


## Panel Dashboard

Combine all of the pie charts into a single dashboard view using Panel.

In [385]:
# Assign Content to dashboard

industry_analysis=pn.Column(industry_value_pie, industry_contribution_pie, industry_shares_pie)
sector_analysis=pn.Column(sector_value_pie, sector_contribution_pie, sector_shares_pie)
current_holdings=pn.Column(holdings_value_pie, holdings_contribution_pie, holdings_shares_pie)

In [386]:
# Create the Tabs

dashboard=pn.Tabs(("Industry Title", industry_analysis), 
                  ("Sector Title", sector_analysis), 
                  ("Current Holdings Title", current_holdings))

In [387]:
# Serve the Panel Dashboard

dashboard.servable()

Tabs
    [0] Column
        [0] Plotly(Figure)
        [1] Plotly(Figure)
        [2] Plotly(Figure)
    [1] Column
        [0] Plotly(Figure)
        [1] Plotly(Figure)
        [2] Plotly(Figure)
    [2] Column
        [0] Plotly(Figure)
        [1] Plotly(Figure)
        [2] Plotly(Figure)

# Interactive Visualization

Combine all of the pie charts into a dropdown view using Panel.

In [346]:
# # Define function to choose a Chart View

# def choose_pie_chart(Views):
    
#     industry_analysis_group=pn.Column(industry_value(), industry_contribution(), industry_shares())
#     sector_analysis_group=pn.Column(sector_value(), sector_contribution(), sector_shares())
#     current_holdings_group=pn.Column(holdings_value(), holdings_contribution(), holdings_shares())
    
#     def industry_contribution():
#         industry_contribution_pie=px.pie(industry_holdings, values="Cumm Contribution", names="Industry", title="Contributions by Industry")
#         return industry_contribution_pie

#     def industry_value():
#         industry_value_pie=px.pie(industry_holdings, values="Daily Balance", names="Industry", title="Current Value by Industry")
#         return industry_value_pie

#     def industry_shares():
#         industry_shares_pie=px.pie(industry_holdings, values="Total Shares", names="Industry", title="Shares Owned by Industry")
#         return industry_shares_pie

#     def sector_contribution():
#         sector_contribution_pie=px.pie(sector_holdings, values="Cumm Contribution", names="Sector", title="Contributions by Sector")
#         return sector_contribution_pie

#     def sector_value():
#         sector_value_pie=px.pie(sector_holdings, values="Daily Balance", names="Sector", title="Current Value by Sector")
#         return sector_value_pie

#     def sector_shares():
#         sector_shares_pie=px.pie(sector_holdings, values="Total Shares", names="Sector", title="Shares Owned by Sector")
#         return sector_shares_pie

#     def holdings_contribution():
#         holdings_contribution_pie=px.pie(current_industry_holdings, values="Cumm Contribution", names="Ticker", title="Contributions by Stock")
#         return holdings_contribution_pie

#     def holdings_value():
#         holdings_value_pie=px.pie(current_industry_holdings, values="Daily Balance", names="Ticker", title="Current Value by Stock")
#         return holdings_value_pie

#     def holdings_shares():
#         holdings_shares_pie=px.pie(current_industry_holdings, values="Total Shares", names="Ticker", title="Shares Owned by Stock")
#         return holdings_shares_pie

#     return interact(choose_pie_chart, Views=pie_chart_list)

# # Call the choose_pie_chart function to return plots with interactivity

# pie_chart_list=[industry_analysis_group, sector_analysis_group, current_holdings_group]

# interact(choose_pie_chart, Views=pie_chart_list)


In [ ]:
# # Old pie chart configurations

# industry_contribution_pie2=industry_holdings.plot(kind="pie", y="Cumm Contribution", figsize=(10, 10), legend=True, title="Contributions by Industry", autopct='%1.1f%%')
# industry_shares_pie2=industry_holdings.plot(kind="pie", y="Total Shares", figsize=(10, 10), legend=True, title="Shares Owned by Industry", autopct='%1.1f%%')
# industry_value_pie2=industry_holdings.plot(kind="pie", y="Daily Balance", figsize=(10, 10), legend=True, title="Current Value by Industry", autopct='%1.1f%%')

# sector_contribution_pie=sector_holdings.plot(kind="pie", y="Cumm Contribution", figsize=(10, 10), legend=True, title="Contributions by Sector", autopct='%1.1f%%')
# sector_shares_pie=sector_holdings.plot(kind="pie", y="Total Shares", figsize=(10, 10), legend=True, title="Shares Owned by Sector", autopct='%1.1f%%')
# sector_value_pie=sector_holdings.plot(kind="pie", y="Daily Balance", figsize=(10, 10), legend=True, title="Current Value by Sector", autopct='%1.1f%%')

# holdings_contribution_pie=current_holdings_df.plot(kind="pie", labels=current_holdings_df["Ticker"], y="Cumm Contribution", figsize=(10, 10), legend=True, title="Contributions by Stock", autopct='%1.1f%%')
# holdings_shares_pie=current_holdings_df.plot(kind="pie", labels=current_holdings_df["Ticker"], y="Total Shares", figsize=(10, 10), legend=True, title="Shares Owned by Stock", autopct='%1.1f%%')
# holdings_value_pie=current_holdings_df.plot(kind="pie", labels=current_holdings_df["Ticker"], y="Daily Balance", figsize=(10, 10), legend=True, title="Current Value by Stock", autopct='%1.1f%%')

# Create Ticker Stats Panel
Using the portfolio database, determine the number of unique stocks and pull stock history.
* Inputs:
    * Transposed Ticker Stat Database (ticker_stats_df_T) (Tickers as Columns)
* Outputs:
    * Ticker Stat Panel (stats_panel)

In [43]:
# Define function that will change negative numbers to red text
def color_negative_red(value):
    if value < 0:
        color='red'
    else:
        color='black'
    return 'color: %s' % color

# Define function that will highlight the max value
def highlight_max(value):
    is_max = value == value.max()
    return ['background-color: yellow' if v else '' for v in is_max]

In [44]:
#Format ticker_stats_df_T to:
# Highlight negative values red
# Highlight larget value yellow
# Add '$' and ',' to currency
# Change percentages to %
# Change floats to 2 decimal places
formatted_stats_df = ticker_stats_df_T.style.applymap(
    color_negative_red, subset=['Beta', 'Cash', 'PE Ratio', 'Profit Margin']
    ).apply(
    highlight_max, subset=['Beta', 'Cash', 'Last Dividend', 'PE Ratio', 'Profit Margin']
    ).format(
    {'Beta':'{:,.2f}', 'Cash':'${:,}', 'Last Dividend':'${:,.2f}', 'PE Ratio':'{:,.2f}', 'Profit Margin':'{:,.1%}'}
    )
formatted_stats_df

,PE Ratio,Profit Margin,Cash,Beta,Last Dividend,Industry,Sector
AAPL,20.81,21.2%,"$48,844,000,000",1.23,$3.08,Consumer Electronics,Technology
GOOG,14.70,21.2%,"$18,498,000,000",1.02,$0.00,Internet Content & Information,Communication Services
TSLA,-131.58,-3.5%,"$6,268,000,000",0.58,$0.00,Auto Manufacturers,Consumer Cyclical


In [45]:
stats_panel = pn.panel(formatted_stats_df)
stats_panel

HTML(Styler)

In [46]:
# code below is my scratchpad area, ignore

In [49]:
# for row in pf:
    if row[index] == 'B':
        lll
    else:
        lll
return lll

In [48]:
stock_dict_copy2=pd.DataFrame(stock_dict_copy)
stock_dict_copy2.to_csv('stock_dict_copy.csv')